# Demo the Liquidator Agent Logic

This notebook:

1. Fetches the latest crvusd contract data into Python objects using `crvusdsim`.
2. Fetches prices and `ExternalMarket`s from the `baseline` scenario.
3. Instantiates a liquidator agent.
4. Checks if there are users to liquidate in the `Controller` and liquidates them.

In [1]:
from crvusdsim.pool import get
from src.agents.liquidator import Liquidator
from src.sim.scenario import Scenario

%load_ext autoreload
%autoreload 2

In [2]:
liquidator = Liquidator()

ETH_POOL = "weth"
(
    pool,
    controller,
    collateral_token,
    stablecoin,
    aggregator,
    stableswap_pools,
    peg_keepers,
    policy,
    factory,
) = get(ETH_POOL, bands_data="controller")

In [3]:
config = "../src/configs/scenarios/baseline.json"
scenario = Scenario(config)
markets = scenario.generate_markets()
prices = scenario.generate_pricepaths(
    "../src/configs/prices/1h_1694894242_1700078242.json"
)

[INFO][22:08:16][root]-433001: Reading price config from ../src/configs/scenarios/baseline.json.


[INFO][22:08:16][root]-433001: Fetching 1inch quotes.
[INFO][22:08:23][root]-433001: We have 372960 quotes.
[INFO][22:08:23][root]-433001: Fitting external markets against 1inch quotes.
[INFO][22:08:23][root]-433001: Reading price config from ../src/configs/prices/1h_1694894242_1700078242.json.


In [4]:
# Set External Market Prices
sample = prices[-1].prices
for in_token in markets:
    for out_token in markets[in_token]:
        markets[in_token][out_token].update_price(sample[in_token][out_token])

In [5]:
liquidator.set_paths(controller, stableswap_pools, markets)
# Artificially raise price if necessary to make liquidation profitable
liquidator.paths[0].collat_pool.update_price(5000)

In [10]:
profits, underwater_debt = liquidator.perform_liquidations(controller)

[INFO][22:07:23][root]-432808: There are 19 users to liquidate.
[INFO][22:07:23][root]-432808: Liquidating user 0xec718904654b29f1f571290259c324fd2e3aa63e with expected profit: [77025.85129377].
[INFO][22:07:23][root]-432808: Executing trade Swap(pool=<SimCurveStableSwapPool address=0x4dece678ceceb27446b35c672dc7d61f30bad69e chain=mainnet>, i=0, j=1, amt=53153255322).
[INFO][22:07:23][root]-432808: Executing trade Liquidation(controller=<crvusdsim.pool.sim_interface.sim_controller.SimController object at 0x7f5cd7c818b0>, position=<crvusdsim.pool.crvusd.controller.Position object at 0x7f5cd89518d0>, amt=53362138823577594494976, frac=1000000000000000000, i=0, j=1).
[INFO][22:07:23][root]-432808: Executing trade Swap(pool=<src.modules.market.ExternalMarket object at 0x7f5cd52d2a90>, i=0, j=1, amt=26049758367989198848).
[INFO][22:07:23][root]-432808: Liquidated user 0xec718904654b29f1f571290259c324fd2e3aa63e with profit: [77025.85129377].
[INFO][22:07:23][root]-432808: Liquidating user 0xb

[INFO][22:07:23][root]-432808: Liquidating user 0xad34c2d6d0e979d6452caf890719038ed27aa0f3 with expected profit: [1178.62667493].
[INFO][22:07:23][root]-432808: Executing trade Swap(pool=<SimCurveStableSwapPool address=0x4dece678ceceb27446b35c672dc7d61f30bad69e chain=mainnet>, i=0, j=1, amt=1548380185).
[INFO][22:07:23][root]-432808: Executing trade Liquidation(controller=<crvusdsim.pool.sim_interface.sim_controller.SimController object at 0x7f5cd7c818b0>, position=<crvusdsim.pool.crvusd.controller.Position object at 0x7f5cd605a0d0>, amt=1554374195494622330880, frac=1000000000000000000, i=0, j=1).
[INFO][22:07:23][root]-432808: Executing trade Swap(pool=<src.modules.market.ExternalMarket object at 0x7f5cd52d2a90>, i=0, j=1, amt=545693666154797952).
[INFO][22:07:23][root]-432808: Liquidated user 0xad34c2d6d0e979d6452caf890719038ed27aa0f3 with profit: [1178.62667493].
[INFO][22:07:23][root]-432808: Liquidating user 0xac01ec664c1062234a73574d44523c906a3a6369 with expected profit: [2352717